# Pricing of a simple bond vanilla

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import newton

df_spot_rate = pd.DataFrame({"SpotRates": [0.025, 0.026, 0.0265,0.028, 0.03,0.033,0.037,0.039,0.043,0.051],
                            "Year": np.arange(1,11,1)}, index=np.arange(1,11,1))
df_spot_rate_constant = pd.DataFrame({"SpotRates": np.repeat(0.05, 10), "Year": np.arange(1,11,1)}, index=np.arange(1,11,1))

def bond_cash_flow(notional, cp, cp_frequency, maturity):
    """
    function to create cash-flow of vanilla bond.

    notional: the bond notional
    cp: the coupon rate as 0.05 for 5% 
    cp_frequency: the yearly frequency of payment? 1 for annual, 2 for semi-annual,...
    maturity: the bond maturity in years
    """
    coupon_cf = np.full(maturity * cp_frequency, cp / cp_frequency * notional)
    principal_cf = np.zeros(maturity * cp_frequency); principal_cf[-1] = notional
    cash_flow = coupon_cf + principal_cf
    time_to_receip_cf = np.arange(1/cp_frequency, maturity + 1 / cp_frequency, 1 / cp_frequency)
    time_cf = pd.Index(np.arange(1,maturity * cp_frequency + 1,1))
    
    cash_flow_df = pd.DataFrame(data = {
        "TimetoReceipt": time_to_receip_cf,
        "Coupon":coupon_cf,
        "Principal":principal_cf,
        "CashFlow":cash_flow
        
    }, index=time_cf)
    return cash_flow_df

def discount_cash_flow(cash_flow, spot_rates):
    """
    Function that takes a dataframe of cash flow and a dataframe of discount factor and returns
    a dataframe of discounted cash flow.
    """
    df_discount_cash_flow = pd.merge(cash_flow[["TimetoReceipt", "CashFlow"]], spot_rates, left_index=True, right_index=True, how="left")
    
    # df_discount_cash_flow["InterpolatedSpotRate"] = df_discount_cash_flow["SpotRate"].interpolate(method="linear",limit_direction="both")
    
    df_discount_cash_flow = cash_flow[["TimetoReceipt", "CashFlow"]].copy()
    df_discount_cash_flow["InterpolatedSpotRates"] = np.interp(
        cash_flow["TimetoReceipt"],
        spot_rates["Year"],
        spot_rates["SpotRates"]
    )
    df_discount_cash_flow["DiscountFactor"] = 1 / (1 + df_discount_cash_flow["InterpolatedSpotRates"])**df_discount_cash_flow["TimetoReceipt"]
    df_discount_cash_flow["DiscountedCF"] = df_discount_cash_flow["CashFlow"] * df_discount_cash_flow["DiscountFactor"]
    return df_discount_cash_flow

def calculate_ytm(df, price=100, frequency=2):
    """
    Calculates the Yield to Maturity (YTM) of a bond.
    
    Parameters
    ----------
    df: pandas.DataFrame
        DataFrame with a "CashFlow" column (bond cash flows)
        and index = timing of cash flows (e.g., years or periods).
    price: float 
        Observed market price of the bond.
    guess: float (default=0.05) 
        initial guess for the numerical solver.
    
    Returns
    -------
    ytm: float
        Yield to Maturity in decimal (e.g., 0.05 = 5%)
    """

    def present_value(ytm):
        periods = df.index.values
        cashflows = df["CashFlow"].values
        return np.sum(cashflows / (1 + ytm/frequency)**periods) - price
    
    # Solve for the YTM that zeroes the function
    ytm = newton(present_value, x0=0.05)
    return ytm

def build_discount_factors(times_to_receipt, df_spot, frequency=1, quotation_type= "BEY"):
    """
    Build discount factors from spot rates and requested times.

    Parameters
    ----------
    times_to_receipt: array-like
        Time points where discount factors are needed.
    df_spot: DataFrame
        Must contain "Year", "SpotRate"
    compounding: str
        "annual" or "continuous"
    quotation_type: str
        EAY: effective annual yield
        BEY: bond equivalent yield
    
    Returns
    -------
    DataFrame with ["TimetoReceipt", "SpotRates", "DiscountFactor"]
    """
    # Interpolate spot rates
    spot_interp = np.interp(times_to_receipt, df_spot["Year"], df_spot["SpotRates"])
    
    # calculate the discount factor for the specified quotation
    if quotation_type == "BEY":
        times = np.arange(1,len(times_to_receipt) + 1,1)
        discount_factors = 1 / (1 + spot_interp/frequency) ** times
    else:
        times = times_to_receipt.copy()
        discount_factors = 1 / (1 + spot_interp) ** times
    
    return pd.DataFrame({
        "TimeToReceipt": times_to_receipt,
        "SpotRate": spot_interp,
        "DiscountFactor": discount_factors
    }, index=np.arange(1, len(times_to_receipt) + 1, 1))
    
def estimated_ytm(price, notional, cp, frequency, maturity):
    """
    Estimate the yield to maturity of a bond

    """
    return (cp * notional + (notional-price)/maturity)/((notional + price)/frequency)

# -----------------------------------------------------------------------------------------
"""
        assess the risk of a fixed income position

"""
# -----------------------------------------------------------------------------------------

def wal(cash_flow_df):
    """
    return the weighted average life of a principal over the life of an investment.
    WAl it measures the average time over which the investor may expect the return of his principal.
    """
    return (cash_flow_df["Principal"] * cash_flow_df.index).sum() / cash_flow_df["Principal"].sum()

def mac_dur(cash_flow_df, spot_price, frequency = 1):
    """
    return the Macaulay Duration Calculation for a cash flow DataFrame

    """
    YtoM =  calculate_ytm(cash_flow_df, spot_price)
    nb_rows = len(cash_flow_df) + 1 
    df_YtoM = pd.DataFrame(
    { "Year": np.arange(1, nb_rows, 1),
     "SpotRates": np.repeat(ytm_b1, nb_rows - 1)
    }, index=np.arange(1, nb_rows, 1))

    # we discount the cash flow with the yield to maturity
    df_discount_factor = build_discount_factors(cash_flow_df["TimetoReceipt"].values,df_YtoM, frequency)
    df_discount_cash_flow = pd.merge(
        cash_flow_df[["TimetoReceipt", "CashFlow"]], 
        df_discount_factor[["SpotRate", "DiscountFactor"]], 
        left_index=True, right_index=True, how="left")
    df_discount_cash_flow["PV"] = df_discount_cash_flow["CashFlow"] * df_discount_cash_flow["DiscountFactor"]

    mac_dur_value =  np.dot(df_discount_cash_flow["PV"], df_discount_cash_flow["TimetoReceipt"]) / np.sum(df_discount_cash_flow["PV"])
    
    return df_discount_cash_flow, mac_dur_value

In [27]:
b1 = bond_cash_flow(100, 0.03, 2, 10)

ytm_b1 = calculate_ytm(b1,100)

df_spot_rate_constant = pd.DataFrame(
    {"SpotRates": np.repeat(ytm_b1, 10), 
     "Year": np.arange(1,11,1)
    }, index=np.arange(1,11,1))

detail, macdur = mac_dur(b1, 100, 2)
print(detail)
print(macdur)

    TimetoReceipt  CashFlow  SpotRate  DiscountFactor         PV
1             0.5       1.5      0.03        0.985222   1.477833
2             1.0       1.5      0.03        0.970662   1.455993
3             1.5       1.5      0.03        0.956317   1.434475
4             2.0       1.5      0.03        0.942184   1.413276
5             2.5       1.5      0.03        0.928260   1.392390
6             3.0       1.5      0.03        0.914542   1.371813
7             3.5       1.5      0.03        0.901027   1.351540
8             4.0       1.5      0.03        0.887711   1.331567
9             4.5       1.5      0.03        0.874592   1.311888
10            5.0       1.5      0.03        0.861667   1.292501
11            5.5       1.5      0.03        0.848933   1.273400
12            6.0       1.5      0.03        0.836387   1.254581
13            6.5       1.5      0.03        0.824027   1.236041
14            7.0       1.5      0.03        0.811849   1.217774
15            7.5       1

# Modeling RMBS cash flow

In [28]:

def mortgage_schedule_nn_prepayment(principal, annual_rate, years, payments_per_year=12):
    """
        Returns the cash flow schedule of a mortgage loan

    """
    r = annual_rate / payments_per_year
    n = years * payments_per_year
    payment = principal * r / (1 - (1 + r) ** -n)
    CPR = 0.02
    schedule = []
    balance = principal
    
    for i in range(1, n + 1):
        interest = balance * r
        scheduled_principal = payment - interest
        prepayment = CPR * balance
        total_principal = scheduled_principal + prepayment
        scheduled_balance = get_scheduled_balance(payment, r, n, i, payments_per_year)
        balance -= total_principal
        schedule.append(
            [i, round(payment,3), round(interest,3), round(scheduled_principal,3), 
             round(prepayment,3), round(total_principal,3), 
             round(balance,3) if balance > 0 else 0, round(scheduled_balance,3)])
    
    cash_flow_df = pd.DataFrame(
        schedule, 
        columns=["Period", "Payment", "Interest", "ScheduledPrincipal",  
                 "Prepayment", "TotalPrincipal", "EndingBalance", "ScheduledBalance"], 
        index = np.arange(1, years * payments_per_year + 1, 1))
    
    return cash_flow_df

def get_scheduled_balance(paiement, rate, term, actual_term, period = 12):
    """
    paiement: constant paiement
    rate: annual rate
    term: the maturity of the loan
    actual_term: where we are in the loan life
    period: number of paiements per year.
    """
    return paiement/rate * (1 - (1 + rate)**(-(term-actual_term))) 
    

def SMM(loan_cf):
    pass
    #smm_df = loan_cf[] loan_cf[EndingBalance] / loan_cf[]

In [29]:
mtg = mortgage_schedule_nn_prepayment(200000, 0.045, int(360/12))
print(mtg.head(12))

    Period   Payment  Interest  ScheduledPrincipal  Prepayment  \
1        1  1013.371   750.000             263.371    4000.000   
2        2  1013.371   734.012             279.358    3914.733   
3        3  1013.371   718.285             295.086    3830.851   
4        4  1013.371   702.812             310.558    3748.332   
5        5  1013.371   687.591             325.779    3667.154   
6        6  1013.371   672.618             340.753    3587.296   
7        7  1013.371   657.888             355.483    3508.735   
8        8  1013.371   643.397             369.974    3431.450   
9        9  1013.371   629.142             384.229    3355.422   
10      10  1013.371   615.118             398.253    3280.629   
11      11  1013.371   601.322             412.049    3207.051   
12      12  1013.371   587.750             425.620    3134.669   

    TotalPrincipal  EndingBalance  ScheduledBalance  
1         4263.371     195736.629        199736.629  
2         4194.091     191542.539

In [30]:
print(get_scheduled_balance(1013.37062, 0.045, 360, 1))
print(get_scheduled_balance(1013.37062, 0.045, 360, 2))

22519.344022184967
22519.34388318329


# modeling prepayment

## Single Monthly Mortality Rate SMM

The monthly prepayment rate, or single monthly mortality rate (SMM) measures the percentage of a pool’s principal balance that has prepaid in the current month. It is based on the change in thepool’s factor (survival factor) from one period to the next:

In [31]:
def SMM(cash_flow):
    smm_df = (cash_flow["ScheduledBalance"] - cash_flow["EndingBalance"]) / cash_flow["ScheduledBalance"]
    smm_df.index = cash_flow.index
    return smm_df

In [32]:
print(SMM(mtg).head(10))

1     0.020026
2     0.039754
3     0.059186
4     0.078329
5     0.097187
6     0.115764
7     0.134065
8     0.152094
9     0.169855
10    0.187353
dtype: float64


In [2]:
par_rates = [0.025, 0.031680, 0.03704]
df_par_rate = pd.DataFrame({"ParRates": par_rates,
                             "TimetoMaturity": np.arange(1,len(par_rates) +1,1)},
                           index=np.arange(1,len(par_rates)+1,1))

def get_df_factor(par_rates):
    discount_factors = pd.DataFrame(
        data={"TimetoMaturity": par_rates["TimetoMaturity"], 
              "DiscountFactors" : np.zeros(par_rates.shape[0])},
        index = par_rates.index
        )
    discount_factors["DiscountFactors"] = 1 / (1 + par_rates["ParRates"])**(par_rates["TimetoMaturity"])
    return discount_factors


def get_spot_rates(par_rates):
    par_rates_discount_factor = get_df_factor(par_rates)
    spot_rates = []
    for i,n in enumerate(par_rates["ParRates"]):
        if i == 0:
            spot_rates.append(n)
        else:
            rate = 1+n
            prod = par_rates_discount_factor["DiscountFactors"].iloc[:i-1].sum()
            spot_rates.append(
                ((1+n)/(1-n*par_rates_discount_factor["DiscountFactors"].iloc[:i].sum()))**(1/par_rates["TimetoMaturity"].iloc[i]) - 1
            )
    spot_rates_df = pd.DataFrame(
        data = {
            "TimetoMaturity": par_rates_discount_factor["TimetoMaturity"],
            "SpotRates": spot_rates
        },
        index= df_par_rate.index
    )
    return spot_rates_df
        
def get_forward_rates_1y(spot_rates: pd.DataFrame) -> pd.DataFrame:
    """
    Build the 1-year forward rate curve from a spot rate curve.
    The first element corresponds to the 1-year spot rate,
    and the next ones are forward 1y in 1y, 1y in 2y, etc.
    """
    
    df_forward = pd.DataFrame(columns=["TimetoMaturity", "ForwardRates"])
    
    # First rate = spot 1y
    first_rate = spot_rates.iloc[0]["SpotRates"]
    df_forward.loc[0] = [spot_rates.iloc[0]["TimetoMaturity"], first_rate]
    
    # Compute 1y forward rates
    for t in range(1, len(spot_rates)):
        s_t = spot_rates.iloc[t-1]["SpotRates"]
        s_t1 = spot_rates.iloc[t]["SpotRates"]
        tenor_t = spot_rates.iloc[t-1]["TimetoMaturity"]
        tenor_t1 = spot_rates.iloc[t]["TimetoMaturity"]
        
        f_t1 = ((1 + s_t1)**tenor_t1 / (1 + s_t)**tenor_t) - 1
        df_forward.loc[t] = [tenor_t1, f_t1]
    return df_forward

spot_rates= get_spot_rates(df_par_rate)
forward_rates = get_forward_rates_1y(spot_rates)


In [43]:
##
## build tree forward rates
##
from math import exp
import pandas as pd
import numpy as np
from scipy.optimize import minimize, minimize_scalar

par_rates = [0.025, 0.031680, 0.03704]
df_par_rate = pd.DataFrame({"ParRates": par_rates,
                             "TimetoMaturity": np.arange(1,len(par_rates) +1,1)},
                           index=np.arange(1,len(par_rates)+1,1))

# ----------------------------------------------------------------------------------------------------
#                               build binomial tree under log normal law
# ----------------------------------------------------------------------------------------------------

# === 1. Construction des nœuds lognormaux ===
def log_normal_node(base_rate: float, sigma: float, node_nb: int) -> list:
    """ Create a node for interest rate binomial model defined by lognormal
        probability.

    Args:
        base_rate (float): first rate in the node number.
        sigma (float): the volatility of interest rate
        node_nb (int): the ith node as the first one begin at 0.

    Returns:
        list: a list of the ith node.
    """
    node = [base_rate]
    for i in range(1, node_nb + 1):
        node.append(node[i-1]*exp(2*sigma))
    return node

# === 2. Construction des cash flows d’un bond ===
def bond_tree_cf(coupon: float, maturity: int, notional: int = 100) -> list:
    tree = []
    # Build tree iteratively
    for t in range(maturity):
        tree.append(list(np.repeat(coupon*notional, t+1)))
    tree.append(list(np.repeat(notional*(1+coupon), maturity+1)))
    return tree

# === 3. Discounting backward ===
def discount_binomial_tree(cash_flow_tree: list, forward_rates_tree: list, risk_neutral_prob: float)-> list:
    
    # error management - to do
    n_periods_rates = len(forward_rates_tree)
    n_periods_cf = len(cash_flow_tree)
    
    # Initialize value tree as a deep copy of cash flows
    value_tree = [list(row) for row in cash_flow_tree]
    
    # calculate the last value of the tree
    for i in range(len(forward_rates_tree[n_periods_rates-1])):
        value_tree[n_periods_rates-1][i] = value_tree[n_periods_rates-1][i]/(1+forward_rates_tree[n_periods_rates-1][i])
    
    # Start from the last period (terminal nodes)
    for t in range(n_periods_rates-2, -1, -1):  # backward
        for i in range(len(forward_rates_tree[t])):
            r = forward_rates_tree[t][i]
            V_up = value_tree[t+1][i+1] 
            V_down = value_tree[t+1][i]
            expected_next = risk_neutral_prob * V_up + (1 - risk_neutral_prob) * V_down
            value_tree[t][i] = (cash_flow_tree[t][i] + expected_next) / (1 + r)
    
    # Return price and full tree
    return value_tree

# === 4. get the present value of the discounted tree ===
def present_value_binomial_tree(cash_flow_tree: list, forward_rates_tree: list, risk_neutral_prob: float)-> float:
    discounted_tree = discount_binomial_tree(cash_flow_tree, forward_rates_tree, risk_neutral_prob)
    return discounted_tree[0][0]

# === 5. Ajout d’un nouveau nœud à l’arbre ===
def build_next_node(next_lower_rate: float, rates_tree: list, sigma: float) -> list:
    if isinstance(rates_tree, (float, int)):
        value_tree = [[float(rates_tree)]]
    elif isinstance(rates_tree, list):
        if all(isinstance(x, (float, int)) for x in rates_tree):
            # Cas: [0.025, 0.03]
            value_tree = [list(map(float, rates_tree))]
        elif all(isinstance(x, list) for x in rates_tree):
            # Cas: [[0.025], [0.03, 0.0366]]
            value_tree = [list(node) for node in rates_tree]
        
    nb_period = len(value_tree)
    node = log_normal_node(next_lower_rate, sigma, nb_period)
    value_tree.append(node)
    return value_tree

# === 6. Fonction objectif ===
def objective_binomial(next_rate: float, rates_tree: list, par_rate: float, sigma: float, risk_neutral_prob: float) -> float:
    new_tree = build_next_node(next_rate, rates_tree, sigma)
    nb_period = len(new_tree)
    cf_bond = bond_tree_cf(par_rate, nb_period-1)
    pv_bond = present_value_binomial_tree(cf_bond, new_tree, risk_neutral_prob)
    return abs(100 - pv_bond)

# === 6. Calibration de l’arbre complet ===
def calibrate_forward_tree(par_rates: list, sigma: float, notional: int = 100):
    """
    Calibrate forward rates tree node-by-node by minimizing abs(100 - PV) for each maturity.
    Returns the forward_rates_tree (list of lists) and list of absolute pricing errors.
    """
    n = len(par_rates)
    # initialize with first par rate as the root short rate
    forward_tree = [[float(par_rates[0])]]
    errors = [0.0]  # error at t=1 should be zero (par1 == spot1)
    
    # iterate maturities 2..n
    for t in range(1, n):
        par_rate_t = par_rates[t]  # par_rates indexed from 0 for 1..n
        # objective wrapper for minimize_scalar
        def obj(x):
            return objective_binomial(x, forward_tree, par_rate_t, sigma, 0.5)
        # optimize next lower rate (search in bounds)
        res = minimize_scalar(obj, bounds=(0.0001, 0.2), method='bounded', options={'xatol':1e-12})
        next_rate = float(res.x)
        # append new node to forward_tree
        forward_tree = build_next_node(next_rate, forward_tree, sigma)
        # compute error and store
        err = obj(next_rate)
        errors.append(err)
    return forward_tree, errors


# ----------------------------------------------------------------------------------------------------
#                               OAS Calculation
# ----------------------------------------------------------------------------------------------------





# ----------------------------------------------------------------------------------------------------
#                               Duration Calculation
# ----------------------------------------------------------------------------------------------------


Calibrated forward rate tree (levels):
t=0:  ['0.025000']
t=1:  ['0.031681', '0.038695']
t=2:  ['0.037041', '0.045242', '0.055258']
Pricing abs errors per maturity: [0.0, 0.8200233550223146, 1.3399734259797356]
t=0:  ['102.114542']
t=1:  ['101.176721', '99.658089']
t=2:  ['100.526430', '99.737705', '98.790984']
t=0:  ['101.540531']
t=1:  ['100.000000', '99.658089']
t=2:  ['100.526430', '99.737705', '98.790984']


In [49]:
# ----------------------------------------------------------------------------------------------------
#                               Payoff functions for binomial tree
# ----------------------------------------------------------------------------------------------------
def vanilla_payoff(value_at_node: float) -> float:
    return value_at_node

def callable_payoff(value_at_node: float, strike: float) -> float:
    return min(value_at_node, strike)

def puttable_payoff(value_at_node: float, strike: float) -> float:
    return max(value_at_node, strike)

# ===  Discounting backward ===
def discount_binomial(
    cash_flow_tree: list, 
    forward_rates_tree: list, 
    payoff_fn: callable,
    event_calendar: list = [], 
    strike: float = 100, 
    risk_neutral_prob: float = 0.5
    )-> list:
    
    # error management - to do
    n_periods_rates = len(forward_rates_tree)
    n_periods_cf = len(cash_flow_tree)
    
    # Initialize value tree as a deep copy of cash flows
    value_tree = [list(row) for row in cash_flow_tree]
    value_at_node: float
    
    # build internal function to manage calendar event
    def apply_payoff_event(t, value):
        if t in event_calendar:
            return payoff_fn(value, strike)
        return value
    
    # calculate the last value of the tree
    for i in range(len(forward_rates_tree[n_periods_rates-1])):
        value_at_node = value_tree[n_periods_rates-1][i]/(1+forward_rates_tree[n_periods_rates-1][i])
        value_tree[n_periods_rates-1][i] = apply_payoff_event(n_periods_rates-1, value_at_node)
    
    # backward induction
    for t in range(n_periods_rates-2, -1, -1): 
        for i in range(len(forward_rates_tree[t])):
            r = forward_rates_tree[t][i]
            V_up = value_tree[t+1][i+1] 
            V_down = value_tree[t+1][i]
            expected_next = risk_neutral_prob * V_up + (1 - risk_neutral_prob) * V_down
            value_at_node = (cash_flow_tree[t][i] + expected_next) / (1 + r)
            value_tree[t][i] =  apply_payoff_event(n_periods_rates-1, value_at_node)
    
    # Return price and full tree
    return value_tree


In [53]:
# ----------------------------------------------------------------------------------------------------
#                               Main
# ----------------------------------------------------------------------------------------------------
if __name__ == "__main__":
    
    # calibrate a binomial tree under log noraml law
    par_rates = [0.04, 0.04, 0.04, 0.04, 0.04]  # 1y,2y,3y par rates
    sigma = 0.15
    tree, errors = calibrate_forward_tree(par_rates, sigma)
    print("Calibrated forward rate tree (levels):")
    for t, level in enumerate(tree):
        print(f"t={t}: ", ["{:.6f}".format(x) for x in level])
    print("Pricing abs errors per maturity:", errors)

    # set up a straight bond
    coupon = 0.045
    cf_bond = bond_tree_cf(coupon, len(tree)-1)
    bond_tree_discounted = discount_binomial(cf_bond, tree, vanilla_payoff )
    print("bond cash flow")
    for t, level in enumerate(bond_tree_discounted):
        print(f"t={t}: ", ["{:.6f}".format(x) for x in level])
    
    # create a callable bond from the above straight bond
    call_calendar = [1,2,3,4,5]
    callable_bond_tree_discounted = discount_binomial(cf_bond, tree, callable_payoff, call_calendar)
    print("callable bond PV per node")
    for t, level in enumerate(callable_bond_tree_discounted):
        print(f"t={t}: ", ["{:.6f}".format(x) for x in level])
        

Calibrated forward rate tree (levels):
t=0:  ['0.040000']
t=1:  ['0.034074', '0.045995']
t=2:  ['0.029051', '0.039214', '0.052934']
t=3:  ['0.024790', '0.033463', '0.045171', '0.060974']
t=4:  ['0.021174', '0.028582', '0.038581', '0.052079', '0.070299']
Pricing abs errors per maturity: [0.0, 0.6096108986059647, 0.5820835814389795, 0.555715583698202, 0.5304681641696902]
bond cash flow
t=0:  ['102.225911']
t=1:  ['104.001955', '99.627940']
t=2:  ['104.502575', '101.588792', '97.831792']
t=3:  ['103.889289', '102.187607', '99.957453', '97.063352']
t=4:  ['102.333226', '101.596226', '100.618054', '99.327150', '97.636252']
callable bond PV per node
t=0:  ['99.904500']
t=1:  ['100.000000', '98.801360']
t=2:  ['100.000000', '100.000000', '97.691388']
t=3:  ['100.000000', '100.000000', '99.661781', '97.063352']
t=4:  ['100.000000', '100.000000', '100.000000', '99.327150', '97.636252']
